In [5]:
from keras import applications
from keras.applications import InceptionV3
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
from keras.utils import to_categorical, pad_sequences
from keras.layers import BatchNormalization, Dropout

from glob import glob

base = InceptionV3(
    include_top=False,
    input_shape=(150, 150, 3)
)

In [6]:
base_dir = './kagglecatsanddogs_5340/PetImages'
file_list_dog = glob(base_dir+'/Dog/*.jpg')[:1000]
file_list_cat = glob(base_dir+'/Cat/*.jpg')[:1000]

# 1000개
file_list = np.concatenate([file_list_dog, file_list_cat], axis=0)
X = np.zeros((len(file_list),150,150,3))
y = []

# 방법 2: data가 없는 파일 제거함.
#         --> 제거 후에 나머지 이미지 파일은 channel 통일시킴
for i, file in enumerate(file_list):
    t = plt.imread(file)
    f = cv.cvtColor(t, cv.COLOR_BGR2RGB)
    f = cv.resize(f, (150,150))
    if 'Cat' in file:
        y.append(0)
    else:
        y.append(1)

    X[i] = f

y = np.array(y)

index = np.random.choice(np.arange(len(X)), len(X), replace=False)
tr_ind = index[:int(index.size*0.8)]
te_idd = index[int(index.size*0.8):]

X_train = X[tr_ind]
y_train = y[tr_ind]

X_test = X[te_idd]
y_test = y[te_idd]

In [8]:
pretrained_X = base.predict(X_train)

50/50 [==============================] - 17s 345ms/step


In [9]:
pretrained_X.shape

(1600, 3, 3, 2048)

In [10]:
model = Sequential(
    [
        Flatten(input_shape=(3,3,2048)),
        Dense(2048, activation='relu'),
        Dense(1024, activation='relu'),
        Dense(1, activation='sigmoid')
    ]
)

opt = 'adam'
loss = 'binary_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=opt,
              loss=loss,
              metrics=metrics)

In [11]:
model.fit(pretrained_X, y_train, epochs=30)

Epoch 1/30
50/50 [==============================] - 12s 240ms/step - loss: 132.8560 - accuracy: 0.5312
Epoch 2/30
50/50 [==============================] - 12s 236ms/step - loss: 4.2518 - accuracy: 0.5744
Epoch 3/30
50/50 [==============================] - 12s 234ms/step - loss: 2.0945 - accuracy: 0.6019
Epoch 4/30
50/50 [==============================] - 12s 238ms/step - loss: 1.0821 - accuracy: 0.6637
Epoch 5/30
50/50 [==============================] - 12s 238ms/step - loss: 0.9680 - accuracy: 0.6606
Epoch 6/30
50/50 [==============================] - 12s 243ms/step - loss: 0.9857 - accuracy: 0.6587
Epoch 7/30
50/50 [==============================] - 12s 243ms/step - loss: 0.6470 - accuracy: 0.7312
Epoch 8/30
50/50 [==============================] - 12s 237ms/step - loss: 0.6676 - accuracy: 0.7350
Epoch 9/30
50/50 [==============================] - 12s 235ms/step - loss: 0.4855 - accuracy: 0.7837
Epoch 10/30
50/50 [==============================] - 12s 231ms/step - loss: 0.5782 - accu

In [ ]:
model.save('cat_or_dog_with_inception.h5')

In [12]:
pretrained_X_test = base.predict(X_test)

13/13 [==============================] - 4s 298ms/step


In [13]:
model.evaluate(pretrained_X_test, y_test)

13/13 [==============================] - 1s 36ms/step - loss: 0.7104 - accuracy: 0.5000


[0.7103828191757202, 0.5]

In [16]:
new_layers = base.layers.extend(model.layers)

new_model = Sequential(new_layers)
new_model.compile(optimizer=opt, loss=loss, metrics=metrics)

In [17]:
new_model.save('cat_or_dog_with_inception.h5')

ValueError: Weights for model sequential_2 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.